In [ ]:
#Crop data by sensor activation (1st frame activated) 

#Read mat files containing video and sensor information
#Crop based on sensor activation 
#Save new mat variables in a new file 


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import os, sys 
import h5py
import hdf5storage

directory = '.'
sensor1 = #int
sensor2 = #int

def one_runs(a):
    # Create an array that is 1 where a is 1, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 1).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges


i_file = 0;
for filename in sorted(os.listdir(directory)):
    if filename.endswith(".mat"): 
        i_file = i_file + 1;
        name = os.path.join(directory, filename)
        f = h5py.File(name, 'r')
        #change to what you want file name to be 
        trial = name[-7:-4]
        print("Analyzing " + str(filename) + "...")
        
        allSensor = []
        allPixel = []
        struArray = f['data/frame']
        shape = struArray['sns'].shape
        shape = shape[0] - 1 

        #shape approximates frames 
        while shape > 0: 
            value = f[struArray['sns'][shape,0]].value 
            pixel = f[struArray['img'][shape,0]].value
            allSensor.append(value)
            allPixel.append(pixel)
            shape = shape - 1 

        frames = np.array([e[0] for e in allSensor])
        numberFrames = (frames.T.shape)[1]
        
        
        #find location of 1 runs
        OnesLoc = []
        for array in frames.T: 
            OnesLoc.append(one_runs(array))

        #find frames where sensors are first activated 
        sensor1 = []
        sensor2 = []
        for i,v in enumerate(OnesLoc):
            if i == sensor1:  
                sensor1.append(numberFrames - v)
            elif i == sensor2: 
                sensor2.append(numberFrames - v)
            else: 
                pass
                
        sensor1 = np.asarray(sensor1)
        sensor2 = np.asarray(sensor2)

        
        if sensor1.size and sensor2.size:  
            cropPixel = allPixel[(numberFrames - sensor2.min()):(numberFrames - sensor1.min())]
            cropSensor = allSensor[(numberFrames - sensor2.min()):(numberFrames - sensor1.min())]
            
            matfiledata = {} # make a dictionary to store the MAT data in

            # *** u prefix for variable name = unicode format
            matfiledata[u'img'] = cropPixel
            matfiledata[u'sns'] = cropSensor
            hdf5storage.savemat(str(trial) + '.mat', matfiledata, format='7.3', oned_as='column')
        
            print("...file analyzed and saved.")
            print(str(int(i_file/49)) + "% complete")
            print("")
            
        else: 
            print("Error: sensor required was not activated. Skipping file.")
            print("")

In [ ]:
os.system('say "program complete"')